# 모니터링과 디버깅

## 모니터링 범위
- 스파크 <strong>잡의 어느 지점에서 오류</strong>가 발생했는지 파악하려면 <strong>스파크 잡을 모니터링</strong>해야함

### 모니터링 대상 컴포넌트
#### 스파크 애플리케이션과 잡
- 클러스터에서 사용자 애플리케이션이 실행되는 <strong>상황을 파악하거나 디버깅하려면 가장 먼저 스파크 모니터링 도구를 사용해야함</strong>
- 스파크 모니터링 도구
  - <strong>스파크 UI</strong>
  - <strong>스파크 로그</strong>
- 스파크 UI와 스파크 로그는 실행 중인 애플리케이션의 <strong>RDD와 쿼리 실행 계획 같은 개념적 수준의 정보를 제공함</strong>

<br>

#### JVM
- 스파크는 <strong>모든 익스큐터를 개별 자바 가상 머신(JVM)에서 실행</strong>함
- <strong>저수준 디버깅</strong>이 필요하다면 JVM도구가 스파크 모니터링 도구보다 유용
- JVM 도구
  - jstack: 스택 트레이스 제공
  - jmap: 힙 덤프 생성
  - jstat: 시계열 통계 리포트 제공
  - jconsole: JVM 속성변수를 시각화된 방식으로 탐색 가능하게 함
  - etc.

<br>

#### OS와 머신
- <strong>JVM은 호스트 OS에서 실행되므로 머신의 상태를 모니터링</strong>해서 정상 작동 중인지 확인하는 것은 매우 중요
- <strong>자원(CPU, 네트웤, I/O 등)</strong>에 대한 모니터링도 함께 해야함
- 이러한 모니터링 요소들은 <strong>클러스터 수준 모니터링 솔루션</strong>에서 확인 가능
  - dstat, iostat, iotop 같은 명령을 사용하면 세밀한 모니터링 가능

<br>

#### 클러스터
- <strong>스파크 애플리케이션이 실행되는 클러스터도 모니터링</strong>해야함
- <strong>모니터링 대상: YARN, 메소스, 스탠드얼론 클러스터 매니저</strong>
- 클러스터 모니터링 솔루션을 활용하면 클러스터가 동작하지 않는 상황을 빠르게 알 수 있음
  - Ganglia, Prometheus 등

## 모니터링 대상
- 모니터링 대상은 크게 두 가지로 나눌 수 있음
  - 실행 중인 사용자 애플리케이션의 <strong>프로세스</strong>(CPU, 메모리 사용률 등)
  - <strong>프로세스 내부</strong>에서의 쿼리 실행 과정(ex) 잡과 태스크)

### 드라이버와 익스큐터 프로세스
- 스파크 애플리케이션을 모니터링할 땐 <strong>드라이버</strong>를 유심히 관찰해야함
 - 드라이버에는 모든 애플리케이션의 <strong>상태가 보관</strong>되어 있으며 안정적으로 실행 중인지 확인 가능
- <strong>익스큐터</strong>의 상태를 파악하는 것도 매우 중요
  - 스파크는 수월한 모니터링을 지원하기 위해 <strong>드롭위자드 메트릭 라이브러리 기반의 메트릭 시스템</strong>을 갖추고 있음
    - [드롭위자드](https://www.dropwizard.io/en/latest/)

### 쿼리, 잡, 스테이지, 태스크
- 특정 쿼리에서 무슨 일이 일어나기 위해 다음 각각의 정보를 알아야함
  - 쿼리
  - 잡
  - 스테이지
  - 테스크
  
<img src="https://rtfmplz.github.io/images/posts/what-is-job-stage-task-in-spark/spark-rdd-partitions-job-stage-tasks.png"/>

## 스파크 로그
- 스파크를 <strong>가장 상세하게 모니터링</strong>하는 방법 중 하나는 로그 파일을 살펴보는 것
- 스파크 애플리케이션의 로그나 스파크 자체의 로그에서 발견된 이상한 이벤트는 <strong>잡의 실패 지점이나 원인 파악</strong>을 도움
- 클러스터에서 스파크를 실행한다면 클러스터 매니저로 파일에 로그를 저장할 수 있음

## 스파크 UI
- 스파크 UI는 실행 중인 애플리케이션과 스파크 워크로드에 대한 <strong>평가지표를 모니터링</strong>할 수 있는 화면 제공
- 스파크 UI 탭
  - Jobs: 스파크 잡에 대한 정보 제공
  - Stages: 개별 스테이지(스테이지의 태스크를 포함)와 관련된 정보 제공
  - Storage: 스파크 애플리케이션에 캐싱된 정보와 데이터 정보 제공
  - Environment: 스파크 애플리케이션의 구성과 설정 관련 정보 제공
  - Executors: 애플리케이션에서 사용 중인 익스큐터의 상세 정보 제공
  - SQL: SQL과 DataFrame을 포함한 구조적 API 쿼리 정보 제공

In [0]:
path = '/FileStore/tables/bin/2010-summary.parquet'
df = spark.read.parquet(path)

In [0]:
display(df.limit(10))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


In [0]:
df.repartition(2).groupby('dest_country_name').count().explain()

== Physical Plan ==
*(3) HashAggregate(keys=[dest_country_name#330], functions=[finalmerge_count(merge count#348L) AS count(1)#343L])
+- Exchange hashpartitioning(dest_country_name#330, 200)
 +- *(2) HashAggregate(keys=[dest_country_name#330], functions=[partial_count(1) AS count#348L])
 +- Exchange RoundRobinPartitioning(2)
 +- *(1) FileScan parquet [DEST_COUNTRY_NAME#330] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/bin/2010-summary.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [0]:
df.repartition(2).groupby('dest_country_name').count().collect()

Out[ 4 ]: [Row(dest_country_name='Russia', count=1),
 Row(dest_country_name='Anguilla', count=1),
 Row(dest_country_name='Paraguay', count=1),
 Row(dest_country_name='Senegal', count=1),
 Row(dest_country_name='Sweden', count=1),
 Row(dest_country_name='Kiribati', count=1),
 Row(dest_country_name='Guyana', count=1),
 Row(dest_country_name='Philippines', count=1),
 Row(dest_country_name='Singapore', count=1),
 Row(dest_country_name='Malaysia', count=1),
 Row(dest_country_name='Fiji', count=1),
 Row(dest_country_name='Turkey', count=1),
 Row(dest_country_name='Germany', count=1),
 Row(dest_country_name='Afghanistan', count=1),
 Row(dest_country_name='Jordan', count=1),
 Row(dest_country_name='Palau', count=1),
 Row(dest_country_name='France', count=1),
 Row(dest_country_name='Turks and Caicos Islands', count=1),
 Row(dest_country_name='Greece', count=1),
 Row(dest_country_name='Taiwan', count=1),
 Row(dest_country_name='British Virgin Islands', count=1),
 Row(dest_country_name='Dominica', count=1),
 Row(dest_country_name='Equatorial Guinea', count=1),
 Row(dest_country_name='Slovakia', count=1),
 Row(dest_country_name='Argentina', count=1),
 Row(dest_country_name='Angola', count=1),
 Row(dest_country_name='Belgium', count=1),
 Row(dest_country_name='Qatar', count=1),
 Row(dest_country_name='Ecuador', count=1),
 Row(dest_country_name='Finland', count=1),
 Row(dest_country_name='Ghana', count=1),
 Row(dest_country_name='Nicaragua', count=1),
 Row(dest_country_name='Peru', count=1),
 Row(dest_country_name='China', count=1),
 Row(dest_country_name='India', count=1),
 Row(dest_country_name='Curacao', count=1),
 Row(dest_country_name='United States', count=131),
 Row(dest_country_name='Malta', count=1),
 Row(dest_country_name='Kuwait', count=1),
 Row(dest_country_name='Marshall Islands', count=1),
 Row(dest_country_name='Chile', count=1),
 Row(dest_country_name='Martinique', count=1),
 Row(dest_country_name='Cayman Islands', count=1),
 Row(dest_country_name='Nigeria', count=1),
 Row(dest_country_name='Bolivia', count=1),
 Row(dest_country_name='Italy', count=1),
 Row(dest_country_name='Suriname', count=1),
 Row(dest_country_name='Netherlands Antilles', count=1),
 Row(dest_country_name='Norway', count=1),
 Row(dest_country_name='Spain', count=1),
 Row(dest_country_name='Cuba', count=1),
 Row(dest_country_name='Guadeloupe', count=1),
 Row(dest_country_name='Denmark', count=1),
 Row(dest_country_name='Barbados', count=1),
 Row(dest_country_name='Ireland', count=1),
 Row(dest_country_name='Thailand', count=1),
 Row(dest_country_name='Morocco', count=1),
 Row(dest_country_name='Panama', count=1),
 Row(dest_country_name='Cape Verde', count=1),
 Row(dest_country_name='Hong Kong', count=1),
 Row(dest_country_name='Ukraine', count=1),
 Row(dest_country_name='Venezuela', count=1),
 Row(dest_country_name='Israel', count=1),
 Row(dest_country_name='Saint Barthelemy', count=1),
 Row(dest_country_name='Iceland', count=1),
 Row(dest_country_name='Saint Kitts and Nevis', count=1),
 Row(dest_country_name='French Polynesia', count=1),
 Row(dest_country_name='South Korea', count=1),
 Row(dest_country_name='Cyprus', count=1),
 Row(dest_country_name='Bonaire, Sint Eustatius, and Saba', count=1),
 Row(dest_country_name='Uruguay', count=1),
 Row(dest_country_name='Mexico', count=1),
 Row(dest_country_name='Aruba', count=1),
 Row(dest_country_name='Georgia', count=1),
 Row(dest_country_name='Saint Vincent and the Grenadines', count=1),
 Row(dest_country_name='The Bahamas', count=1),
 Row(dest_country_name='Guatemala', count=1),
 Row(dest_country_name='Azerbaijan', count=1),
 Row(dest_country_name='Sint Maarten', count=1),
 Row(dest_country_name='Grenada', count=1),
 Row(dest_country_name='Federated States of Micronesia', count=1),
 Row(dest_country_name='Liberia', count=1),
 Row(dest_country_name='Honduras', count=1),
 Row(dest_country_name='Trinidad and Tobago', count=1),
 Row(dest_country_name='Saudi Arabia', count=1),
 Row(dest_country_name='Uganda', coun

### 스테이지 확인
- stage 총 3개
  - <strong>stage1: 파일 스캔 + 파티션 재분배</strong>
    - task 1개
  - <strong>stage2: 파티션별 집계 수행</strong>
    - task 2개(파티션이 두 개 이므로)
  - <strong>stage3: 셔플 파티션</strong>
    - task 200개(기본값)
    
- 태스크 상세 정보 확인
  - Summary Metrics: 다양한 메트릭에 관한 요약 통계
    - 값이 균일한지 봐야함 (균일하지 않다면 눈여겨 봐야함)
  - Aggregated Metrics by Executor: 익스큐터별 통계
    - 특정 익스큐터가 워크로드를 처리하는 데 어려움을 겪고 있는지 판단할 때 유용
  - Show Additional Metrics: 더 상세한 메트릭 정보

### 스파크 REST API
- 사용자 정의 리포트 솔루션을 구축할 때 REST API 사용
- 스파크 UI 외에도 REST API로 스파크의 상태와 메트릭 확인 가능
- 대부분의 REST API는 스파크 UI와 동일한 정보를 제공하지만 SQL 관련 정보는 제공X

### 스파크 UI 히스토리 서버
- 스파크 UI는 SparkContext가 실행되는 동안 사용 가능
- 정상적으로 종료되거나 비정상적으로 종료된 애플리케이션의 정보를 확인하려면 스파크 히스토리 서버를 이용해야함
  - 그 전에 특정 경로에 이벤트 로그를 저장하도록 스파크 애플리케이션을 설정해야함